# Collect All Functions
------------------
This project comes with functions that collect all STIX objects from all ATT&CK Matrices at once. These functions help collect more with less API call requests.

## Import ATTACK API Client

In [1]:
from attackcti import attack_client

## Import Extra Libraries

In [7]:
import pandas
import json

## Initialize ATT&CK Client Variable

In [2]:
lift = attack_client()

## Get All Techniques
We can extract all STIX objects of type `attack-pattern` (technique) across all ATT&CK matrices.
* By default, this function removes `deprecated` and `revoked` techniques. If you want to keep those techniques in the results, you can run the function with the parameter `skip_revoked_deprecated=False`.
* By default, this function also includes all techniques and sub-techniques. If you want to only get techniques that are not sub-techniques, you can run the function with the parameter `include_subtechniques=False`.

In [3]:
techniques = lift.get_techniques()

In [4]:
print("Number of Techniques in ATT&CK")
len(techniques)

Number of Techniques in ATT&CK


858

By default, the data returned by the available functions in the attackcti library is of type **stix2**. However, if you want to interact with libraries such as **Pandas**, it needs to be of type **dict**

In [8]:
all_techniques = []
for t in techniques:
    all_techniques.append(json.loads(t.serialize()))
df = pandas.json_normalize(all_techniques)
df.reindex(['created','name', 'x_mitre_data_sources', 'x_mitre_platforms'], axis=1)[0:5]

,created,name,x_mitre_data_sources,x_mitre_platforms
0,2020-02-20T22:10:20.484Z,Disk Structure Wipe,"[Driver: Driver Load, Drive: Drive Modificatio...","[Linux, macOS, Windows, Network]"
1,2020-10-02T16:39:33.966Z,Gather Victim Host Information,[Internet Scan: Response Content],[PRE]
2,2020-10-02T16:58:58.738Z,Digital Certificates,NaN,[PRE]
3,2020-02-11T18:58:11.791Z,Keylogging,"[Driver: Driver Load, Process: OS API Executio...","[Windows, macOS, Linux, Network]"
4,2024-03-29T16:59:10.374Z,File/Path Exclusions,[File: File Creation],"[Linux, macOS, Windows]"


We can now access the schema of the dataframe

In [9]:
list(df)

['type',
 'spec_version',
 'id',
 'created_by_ref',
 'created',
 'modified',
 'name',
 'description',
 'kill_chain_phases',
 'external_references',
 'object_marking_refs',
 'x_mitre_attack_spec_version',
 'x_mitre_contributors',
 'x_mitre_data_sources',
 'x_mitre_deprecated',
 'x_mitre_detection',
 'x_mitre_domains',
 'x_mitre_impact_type',
 'x_mitre_is_subtechnique',
 'x_mitre_modified_by_ref',
 'x_mitre_platforms',
 'x_mitre_version',
 'x_mitre_defense_bypassed',
 'x_mitre_remote_support',
 'x_mitre_permissions_required',
 'x_mitre_system_requirements',
 'x_mitre_effective_permissions',
 'x_mitre_network_requirements',
 'x_mitre_tactic_type']

Showing one technique example:

In [10]:
techniques[0]

AttackPattern(type='attack-pattern', spec_version='2.1', id='attack-pattern--0af0ca99-357d-4ba1-805f-674fdfb7bef9', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2020-02-20T22:10:20.484Z', modified='2024-10-15T16:32:05.064Z', name='Disk Structure Wipe', description='Adversaries may corrupt or wipe the disk data structures on a hard drive necessary to boot a system; targeting specific critical systems or in large numbers in a network to interrupt availability to system and network resources. \n\nAdversaries may attempt to render the system unable to boot by overwriting critical data located in structures such as the master boot record (MBR) or partition table.(Citation: Symantec Shamoon 2012)(Citation: FireEye Shamoon Nov 2016)(Citation: Palo Alto Shamoon Nov 2016)(Citation: Kaspersky StoneDrill 2017)(Citation: Unit 42 Shamoon3 2018) The data contained in disk structures may include the initial executable code for loading an operating system or the location o

### Enrich Techniques Data Sources
As you might already know, the ATT&CK data model now represents `data sources` as objects. However, when retrieving techniques from ATT&CK TAXII server, their data sources section only includes data sources and data components names. Therefore, we created a parameter that you can use with the `get_techniques()` function to enrich the data sources section of each technique. The parameter `enrich_data_sources` is set to `False` by default.

In [11]:
techniques = lift.get_techniques(enrich_data_sources=True)

This function returns a list of techniques with the `x_mitre_data_sources` attribute as a list of STIX objects representing data sources with their respective data components depending on the technique's detection context.

In [12]:
for ds in techniques[0]['x_mitre_data_sources']:
    for dc in ds['data_components']:
        print(ds['name'], '-', dc['name'])

Driver - Driver Load
Drive - Drive Modification
Drive - Drive Access
Command - Command Execution
Process - Process Creation


In [13]:
techniques[0]['x_mitre_data_sources'][0]

{'id': 'x-mitre-data-source--9ec8c0d7-6137-456f-b829-c5f8b96ba054',
 'modified': '2022-03-30T14:26:51.805Z',
 'created': '2021-10-20T15:05:19.274Z',
 'type': 'x-mitre-data-source',
 'spec_version': '2.1',
 'name': 'Driver',
 'description': 'A computer program that operates or controls a particular type of device that is attached to a computer. Provides a software interface to hardware devices, enabling operating systems and other computer programs to access hardware functions without needing to know precise details about the hardware being used(Citation: IOKit Fundamentals)(Citation: Windows Getting Started Drivers)',
 'x_mitre_modified_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'x_mitre_platforms': ['Linux', 'Windows', 'macOS'],
 'x_mitre_domains': ['enterprise-attack'],
 'x_mitre_version': '1.0',
 'x_mitre_attack_spec_version': '2.1.0',
 'x_mitre_contributors': ['Center for Threat-Informed Defense (CTID)'],
 'x_mitre_collection_layers': ['Host'],
 'created_by_ref': '

## Get All Groups
We can also extract all the available groups across all ATT&CK matrices at once.

In [14]:
groups = lift.get_groups()

In [15]:
print("Number of Groups in ATT&CK")
len(groups)

Number of Groups in ATT&CK


163

In [16]:
groups_list = []
for t in groups:
    groups_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(groups_list)
df[0:4]

,type,spec_version,id,created_by_ref,created,modified,name,description,aliases,external_references,object_marking_refs,x_mitre_attack_spec_version,x_mitre_domains,x_mitre_modified_by_ref,x_mitre_version,x_mitre_deprecated,x_mitre_contributors
0,intrusion-set,2.1,intrusion-set--2e5d3a83-fe00-41a5-9b60-237efc8...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2017-05-31T21:31:46.025Z,2022-04-25T14:00:00.188Z,Moafee,[Moafee](https://attack.mitre.org/groups/G0002...,[Moafee],"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,2.1.0,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.1,NaN,NaN
1,intrusion-set,2.1,intrusion-set--44e43fad-ffcb-4210-abcf-eaaed97...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2019-02-19T16:01:38.585Z,2024-04-11T02:59:52.392Z,APT39,[APT39](https://attack.mitre.org/groups/G0087)...,"[APT39, ITG07, Chafer, Remix Kitten]","[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,3.2,False,NaN
2,intrusion-set,2.1,intrusion-set--44d37b89-a739-4810-9111-0d2617a...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2023-07-05T17:54:54.789Z,2024-04-04T21:24:48.602Z,Scattered Spider,[Scattered Spider](https://attack.mitre.org/gr...,"[Scattered Spider, Roasted 0ktapus, Octo Tempe...","[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,"[enterprise-attack, mobile-attack]",identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2.0,False,NaN
3,intrusion-set,2.1,intrusion-set--44102191-3a31-45f8-acbe-34bdb44...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2020-05-26T14:20:20.623Z,2022-04-25T14:00:00.188Z,Rocke,[Rocke](https://attack.mitre.org/groups/G0106)...,[Rocke],"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,2.1.0,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0,NaN,NaN


## Get All Software
We can extract all Enterprise, Mobile and ICS (Software Malware & Tools)

In [17]:
software = lift.get_software()

In [18]:
print("Number of Software in ATT&CK")
len(software)

Number of Software in ATT&CK


826

In [19]:
software_list = []
for t in software:
    software_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(software_list)
df[0:4]

,type,spec_version,id,created_by_ref,created,modified,name,description,external_references,object_marking_refs,x_mitre_attack_spec_version,x_mitre_domains,x_mitre_modified_by_ref,x_mitre_version,x_mitre_aliases,x_mitre_deprecated,x_mitre_contributors,x_mitre_platforms,is_family
0,tool,2.1,tool--102c3898-85e0-43ee-ae28-62a0a3ed9507,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2017-05-31T21:33:09.047Z,2022-04-25T14:00:00.188Z,UACMe,[UACMe](https://attack.mitre.org/software/S011...,"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,2.1.0,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0,NaN,NaN,NaN,NaN,NaN
1,tool,2.1,tool--294e2560-bd48-44b2-9da2-833b5588ad11,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2017-05-31T21:33:02.863Z,2022-10-12T21:28:49.335Z,ipconfig,[ipconfig](https://attack.mitre.org/software/S...,"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,2.1.0,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.1,[ipconfig],False,NaN,NaN,NaN
2,tool,2.1,tool--26c87906-d750-42c5-946c-d4162c73fc7b,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2019-01-31T01:39:56.283Z,2024-10-07T19:08:53.273Z,Impacket,[Impacket](https://attack.mitre.org/software/S...,"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.7,[Impacket],False,"[Jacob Wilkin, Trustwave, SpiderLabs]","[Linux, macOS, Windows]",NaN
3,tool,2.1,tool--242f3da3-4425-4d11-8f5c-b842886da966,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2017-05-31T21:32:12.684Z,2024-09-12T15:17:22.004Z,Windows Credential Editor,[Windows Credential Editor](https://attack.mit...,"[{'source_name': 'mitre-attack', 'url': 'https...",[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.2.0,[enterprise-attack],identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.1,"[Windows Credential Editor, WCE]",False,NaN,[Windows],NaN


## Get All Relationships
We can also get all relationships from all ATT&CK matrices with one API request.

In [20]:
relationships = lift.get_relationships()

In [21]:
print("Number of Relationships in ATT&CK")
len(relationships)

Number of Relationships in ATT&CK


22142

In [22]:
relations_list = []
for t in relationships:
    relations_list.append(json.loads(t.serialize()))
df = pandas.json_normalize(relations_list)
df[0:4]

,type,spec_version,id,created_by_ref,created,modified,relationship_type,description,source_ref,target_ref,object_marking_refs,x_mitre_attack_spec_version,x_mitre_deprecated,x_mitre_modified_by_ref,x_mitre_version,external_references
0,relationship,2.1,relationship--0005fb3b-274a-4ac1-8fb2-51366fcd...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2023-09-01T21:33:59.394Z,2023-09-01T21:33:59.394Z,mitigates,Consider blocking download/transfer and execut...,course-of-action--21da4fd4-27ad-4e9c-b93d-0b9b...,attack-pattern--43c9bc06-715b-42db-972f-52d25c...,[marking-definition--fa42a846-8d90-4e51-bc29-7...,3.1.0,False,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,0.1,NaN
1,relationship,2.1,relationship--00038d0e-7fc7-41c3-9055-edb4d87e...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2021-04-27T01:56:35.810Z,2021-04-27T01:56:35.810Z,uses,[Explosive](https://attack.mitre.org/software...,malware--6a21e3a4-5ffe-4581-af9a-6a54c7536f44,attack-pattern--707399d6-ab3e-4963-9315-d9d381...,[marking-definition--fa42a846-8d90-4e51-bc29-7...,2.1.0,NaN,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0,[{'source_name': 'CheckPoint Volatile Cedar Ma...
2,relationship,2.1,relationship--00069454-a469-4905-97fd-b4057e86...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2022-03-30T14:26:51.834Z,2022-05-24T14:00:00.188Z,detects,Monitor network data for uncommon data flows. ...,x-mitre-data-component--a7f22107-02e5-4982-906...,attack-pattern--18cffc21-3260-437e-80e4-4ab8bf...,[marking-definition--fa42a846-8d90-4e51-bc29-7...,2.1.0,NaN,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0,NaN
3,relationship,2.1,relationship--0040fdbd-ec7e-49b3-b715-c8c91e08...,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,2017-12-14T16:46:06.044Z,2021-08-27T14:42:00.464Z,uses,Variants of [Emissary](https://attack.mitre.or...,malware--0f862b01-99da-47cc-9bdb-db4a86a95bb1,attack-pattern--9efb1ea7-c37b-4595-9640-b7680c...,[marking-definition--fa42a846-8d90-4e51-bc29-7...,2.1.0,NaN,identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5,1.0,"[{'source_name': 'Emissary Trojan Feb 2016', '..."


## Get All Data Sources
Now that `data sources` are part of the ATT&CK data model as objects, we can retrieve all that information at once.

In [23]:
data_sources = lift.get_data_sources()

In [24]:
print("Number of Data Sources in ATT&CK")
len(data_sources)

Number of Data Sources in ATT&CK


42

In [25]:
data_sources[0]

{'id': 'x-mitre-data-source--2cd6cc81-d86e-4595-a4f0-43f5519f14e6',
 'modified': '2022-04-25T14:00:00.188Z',
 'created': '2021-10-20T15:05:19.271Z',
 'type': 'x-mitre-data-source',
 'spec_version': '2.1',
 'name': 'WMI',
 'description': 'The infrastructure for management data and operations that enables local and remote management of Windows personal computers and servers(Citation: Microsoft WMI System Classes)(Citation: Microsoft WMI Architecture)',
 'x_mitre_modified_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'x_mitre_platforms': ['Windows'],
 'x_mitre_domains': ['enterprise-attack'],
 'x_mitre_version': '1.0',
 'x_mitre_attack_spec_version': '2.1.0',
 'x_mitre_contributors': ['Center for Threat-Informed Defense (CTID)'],
 'x_mitre_collection_layers': ['Host'],
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'external_references': [{'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/datasources/DS0005',
   'external_id': 'DS0005'

## Get All Data Components
Now that `data components` are also part of the ATT&CK data model as objects, we can retrieve all that information at once.

In [26]:
data_components = lift.get_data_components()

In [27]:
print("Number of data components in ATT&CK")
len(data_components)

Number of data components in ATT&CK


119

In [28]:
data_components[0]

{'id': 'x-mitre-data-component--685f917a-e95e-4ba0-ade1-c7d354dae6e0',
 'modified': '2022-10-07T16:14:39.124Z',
 'created': '2021-10-20T15:05:19.273Z',
 'type': 'x-mitre-data-component',
 'spec_version': '2.1',
 'name': 'Command Execution',
 'description': 'The execution of a line of text, potentially with arguments, created from program code (e.g. a cmdlet executed via powershell.exe, interactive commands like >dir, shell executions, etc. )',
 'x_mitre_data_source_ref': 'x-mitre-data-source--73691708-ffb5-4e29-906d-f485f6fa7089',
 'x_mitre_modified_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'x_mitre_deprecated': False,
 'x_mitre_domains': ['enterprise-attack'],
 'x_mitre_version': '1.1',
 'x_mitre_attack_spec_version': '2.1.0',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'revoked': False,
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'external_references': []}